In [ ]:
# 导入必要的库
import mediapipe as mp  # MediaPipe 库用于进行手部检测
import cv2  # OpenCV 库用于处理视频流和图像操作
from math import sqrt  # 数学库，用于计算平方根
from pyfirmata import Arduino, SERVO  # PyFirmata 库用于控制 Arduino
from time import sleep  # 导入 sleep 函数用于延时

# 初始化 Arduino 端口和舵机针脚
port = "com7"  # Arduino 所连接的端口
pins = [10, 11, 12]  # 三个舵机连接的针脚编号
board = Arduino(port)  # 初始化 Arduino 板
for pin in pins:
    board.digital[pin].mode = SERVO  # 将指定针脚设为舵机模式

# 定义旋转舵机的函数
def rotateservo(pins, angle):
    """
    控制舵机旋转到指定角度。

    :param pins: 舵机连接的针脚列表
    :param angle: 旋转的角度
    """
    for pin in pins:
        board.digital[pin].write(angle)  # 控制舵机旋转到指定角度
    sleep(0.015)  # 延时 15 毫秒

# 初始化 MediaPipe 的手部检测和绘图模块
mp_drawing = mp.solutions.drawing_utils  # 用于绘制手部关键点的工具
mp_hands = mp.solutions.hands  # 手部检测模块

# 打开摄像头
video = cv2.VideoCapture(0)  # 初始化摄像头，参数 0 表示默认摄像头

# 设置手部检测的置信度
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.8) as hands:
    click = 0  # 初始化点击计数器

    # 循环读取摄像头数据
    while video.isOpened():
        ret, frame = video.read()  # 从摄像头获取一帧图像
        if not ret:
            break  # 如果没有读取到帧，则退出循环
        
        # 转换图像为 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # 将图像从 BGR 转换为 RGB 格式
        image = cv2.flip(image, 1)  # 水平翻转图像
        imageHeight, imageWidth, _ = image.shape  # 获取图像的高度和宽度
        
        # 处理图像以检测手部
        results = hands.process(image)  # 使用 MediaPipe 处理图像以获取手部位置
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # 将图像转换回 BGR 格式以便显示

        # 检测到手部
        if results.multi_hand_landmarks:
            hands_landmarks = []

            # 遍历检测到的每个手部
            for handLandmarks in results.multi_hand_landmarks:
                hands_landmarks.append(handLandmarks)
                # 绘制手部关键点和连接线
                mp_drawing.draw_landmarks(image, handLandmarks, mp_hands.HAND_CONNECTIONS, 
                                          mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2))

            # 确保检测到两只手
            if len(hands_landmarks) == 2:
                hand1 = hands_landmarks[0]
                hand2 = hands_landmarks[1]

                # 获取手1的食指尖端坐标
                index_finger_tip_1 = hand1.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                index_finger_tip_1 = mp_drawing._normalized_to_pixel_coordinates(index_finger_tip_1.x, 
                                                                                 index_finger_tip_1.y, 
                                                                                 imageWidth, 
                                                                                 imageHeight)

                # 获取手2的食指尖端坐标
                index_finger_tip_2 = hand2.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                index_finger_tip_2 = mp_drawing._normalized_to_pixel_coordinates(index_finger_tip_2.x, 
                                                                                 index_finger_tip_2.y, 
                                                                                 imageWidth, 
                                                                                 imageHeight)

                if index_finger_tip_1 and index_finger_tip_2:
                    # 计算两只手的食指尖端之间的距离
                    distance = sqrt((index_finger_tip_1[0] - index_finger_tip_2[0]) ** 2 + 
                                    (index_finger_tip_1[1] - index_finger_tip_2[1]) ** 2)

                    # 检查距离是否小于一定阈值
                    if distance < 50:  # 阈值可以根据实际情况调整
                        click += 1  # 增加点击计数

                        # 每第六次检测到的点击触发一次舵机旋转
                        if click % 6 == 0:
                            print("Double hands detected, rotating servos")  # 打印消息
                            for i in range(0, 180):  # 舵机从 0 度旋转到 180 度
                                rotateservo(pins, i)  # 调用函数旋转舵机
                            for i in range(180, 0, -1):  # 舵机从 180 度旋转回 0 度
                                rotateservo(pins, i)  # 调用函数旋转舵机
                            break

        # 显示图像窗口
        cv2.imshow('Hand Tracking', image)

        # 检查是否按下 'q' 键以退出
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# 释放视频流
video.release()
cv2.destroyAllWindows()  # 销毁所有窗口
